# README

If no repo is present, the script will download it (it usually takes a while)

* Supply a valid repo filepath for `FILEPATH` variable, ex: `"apps/catalog/models.py"`

For every function present in FILEPATH, you get an object like this:
~~~
{
    'name': 'get_external_order_mapping_list',
    'has_docstring': False,
    'docstring': None,
    'arguments': [{'name': 'order_uuid', 'type': 'UUID, str'}],
    'return_type': 'missing',
    'score': 34.0
}
~~~

In [13]:
import ast
import os
from operator import itemgetter
from apps.patterns.analyzer import BaseFileAnalyzer
from apps.structure import get_full_path

class ServiceDocAnalyzer(BaseFileAnalyzer):
    def __init__(self, *args, **kwargs):
        self.doc_elements = []
        super(ServiceDocAnalyzer, self).__init__(*args, **kwargs)
        
    def build_doc(self, payload):
        self.doc_elements.append(payload)
    
    def get_analysis(self):
        elements = sorted(self.doc_elements, key=itemgetter('name'))
        avg_score = sum(list(map(lambda x: x["score"], elements))) / len(elements)
        return {
            "services": elements,
            "global": {
                "score": avg_score
            }
        }
    
    def get_function_score(self, payload):
        score = 100
        if not payload.get('has_docstring'):
            score -= 33
            
        arguments = payload.get('arguments', [])
        if len(arguments) > 0:
            missing_args = 0
            for arg in arguments:
                if arg.get("type", "missing") == "missing":
                    missing_args += 1
            score_to_remove = 33*(missing_args/len(arguments)) 
            score -= score_to_remove
        
        if payload.get("return_type") == "missing":
            score -= 33
        return score
    
    def get_processed_type(self, annotation):
        if annotation is None:
            return "missing"
        if annotation.__class__ == ast.Name:
            return annotation.id
        if annotation.__class__ == ast.Attribute:
            return annotation.attr
        if annotation.__class__ == ast.Subscript:
            inner_type = self.get_processed_type(annotation.slice)
            outer_type = self.get_processed_type(annotation.value)
            return f"{outer_type}[{inner_type}]"
        if annotation.__class__ == ast.Str:
            return annotation.s
        if annotation.__class__ == ast.Num:
            raise Exception("oops", annotation, dir(annotation))
        if annotation.__class__ == ast.Constant:
            raise Exception("oops", annotation, dir(annotation))
        if annotation.__class__ == ast.NameConstant:
            if annotation.value is None:
                return "None"
            else:
                return annotation.value
        if annotation.__class__ == ast.Index:
            index_type = self.get_processed_type(annotation.value)
            return index_type
        if annotation.__class__ == ast.Tuple:
            types = []
            for expr in annotation.elts:
                elts_type = self.get_processed_type(expr)
                types.append(elts_type)
            return ", ".join(types)
        if annotation.__class__ == ast.Ellipsis:
            return "..."
        if annotation.__class__ == ast.List:
            types = []
            for expr in annotation.elts:
                elts_type = self.get_processed_type(expr)
                types.append(elts_type)
            return ", ".join(types)
        raise Exception("oops", annotation, dir(annotation))
        
    def visit_FunctionDef(self, node):
        arguments = [
            {
                "name": arg.arg,
                "type": self.get_processed_type(arg.annotation),
            }
            for arg in node.args.args
        ]
        docstring = ast.get_docstring(node)
        return_type = self.get_processed_type(node.returns)
        payload = dict(
            name=node.name,
            has_docstring=False if docstring is None or docstring == "" else True,
            docstring=docstring,
            arguments=arguments,
            return_type=return_type
        )
        score = self.get_function_score(payload)
        payload["score"] = score
        self.build_doc(payload)
        
def percentage(num, den):
    return round(num/den * 100, 2)

# Check repo is download first
from apps.patterns.git import repo_exists, clone_repo

if not repo_exists():
    clone_repo()

FILEPATH = "apps/orders/services.py"
full_file_path = get_full_path(file_path=FILEPATH)

anlyzr = ServiceDocAnalyzer(entrypoint=full_file_path)
anlyzr.start()
all_doc = anlyzr.get_analysis()
all_scores = list(map(lambda x: x['score'], all_doc['services']))
print(f"file score: {round(all_doc['global']['score'], 3)}")
print(f"total functions: {len(all_scores)}")

great_score_limit = 90.0
great_scores = list(filter(lambda x: x['score'] >= great_score_limit , all_doc['services']))
print(f"great scores (> {great_score_limit}): {len(great_scores)} ({percentage(len(great_scores), len(all_scores))}%)")
good_score_limit = 67.0
good_scores = list(filter(lambda x: x['score'] > good_score_limit , all_doc['services']))
print(f"good scores (> {good_score_limit}): {len(good_scores)} ({percentage(len(good_scores), len(all_scores))}%)")
low_score_limit = 67.0
super_low_score_limit = 33.0
low_scores = list(filter(lambda x: x['score'] <= low_score_limit , all_doc['services']))
print(f"low scores (<= {low_score_limit}): {len(low_scores)} ({percentage(len(low_scores), len(all_scores))}%)")
super_low_scores = list(filter(lambda x: x['score'] <= super_low_score_limit , all_doc['services']))
print(f"super low scores (<= {super_low_score_limit}): {len(super_low_scores)} ({percentage(len(super_low_scores), len(all_scores))}%)")

without_docstring = list(filter(lambda x: not x['has_docstring'] , all_doc['services']))
print(f"without docstring : {len(without_docstring)} ({percentage(len(without_docstring), len(all_scores))}%)")




file score: 49.421
total functions: 568
great scores (> 90.0): 101 (17.78%)
good scores (> 67.0): 101 (17.78%)
low scores (<= 67.0): 467 (82.22%)
super low scores (<= 33.0): 171 (30.11%)
without docstring : 439 (77.29%)


In [8]:
def analyze_function(doc_data, funct_name: str):
    return tuple(filter(lambda x: x["name"] == funct_name, doc_data.get('services', [])))

analyze_function(all_doc, 'get_external_order_mapping_list')

({'name': 'get_external_order_mapping_list',
  'has_docstring': False,
  'docstring': None,
  'arguments': [{'name': 'order_uuid', 'type': 'UUID, str'}],
  'return_type': 'missing',
  'score': 34.0},)